

# VARIATIONS TEMPORELLES DU BRUIT EN ANTARCTIQUE

---

**Description du Notebook :**
> Dans ce notebook est étudié comment le bruit sismique à évolué avec les différentes mesures sanitaires mises en place par le gouvernement.


---
# Importation des modules

In [2]:
%matplotlib inline

import datetime
import os
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import warnings

from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD

%load_ext autoreload
%autoreload 2
from seismic_noise import *
import seismosocialdistancing

---
# 1. Récupération des données

### Paramètres initiaux

In [4]:
### Paramètres généraux
load_data = False              # Recalculer les PPSDs et le déplacement RMS
# freqs = [(4.0,14.0)]          # Bandes de fréquences à étudier 
st_file = "DATA/st_metadata/stations_antarctic.txt"  # Fichier de stations
noise_input_data = "B:/SeismicNoiseData/"
timeseries_data = "Files/noise_timeseries/"
PPSD_FOLDER  = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/PPSD"
DRMS_FOLDER  = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/DRMS"
MSEED_FOLDER = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/MSEED"
FIG_FOLDER = "RESULTS"
tStart = "2010-01-01"
tEnd = "2011-12-31"

### Conversion des périodes en UTCDateTime
periods = [[UTCDateTime(tStart), UTCDateTime(tEnd)]]

### Récupération des PPSDs et du déplacement RMS

In [5]:
st_name = np.loadtxt(st_file, dtype=str)
# st_name = ["G.CCD.00.HHZ"]   # Station à étudier
dRMS = {}
for st_str in st_name:
    try:
        dataframe = pd.read_csv("{}/{}.csv".format(DRMS_FOLDER, st_str), index_col=0)
        dRMS[st_str] = dataframe
    except:
        pass
st_name = list(dRMS.keys())

In [6]:
dRMS.keys()

dict_keys(['G.CCD.00.BHZ'])

---
# 2. Variations horaires du bruit

## 2.1 Calcul de variations sur toute la période donnée

#### Tags à afficher sur le graphique

In [7]:
# à changer selon ce que l'on veut

tags = [
    ("2020-03-17", 6, "Confinement 1"),
    ("2020-05-11", 6, "Déconfinement"),
    ("2020-10-30", 6, "Confinement 2"),
    ("2020-12-15", 20, "Déconfinement + Couvre-feu 20h"),
    ("2021-01-10", 18, "Couvre feu à 18h"),
    ("2021-04-03", 6, "Confinement 3"),
    ("2021-05-03", 6, "Déconfinement"),
    ("2021-06-09", 23, "Couvre feu à 23h"),
    ("2021-06-30", 6, "Fin du couvre feu")
]

In [8]:
### Calcul de la variation du bruit
k = 1
for st_str in st_name:
    print("{}/{}    {}".format(k, len(st_name), st_str))
    for freq in dRMS[st_str].columns:

        try:
            noise = dRMS[st_str][freq]  # Série temporelle du bruit
        except:
            continue
        t, h, noise_matrix = hourly_var(noise, periods)
        grid = noise_matrix * 1e9  # Données en [nm]

        ### Mise en graphique de la variation du bruit
        plt.figure(figsize=(16, 8))
        plt.pcolormesh(t, h+0.25, grid)
        plt.clim(np.nanquantile(grid, 0.05), np.nanquantile(grid, 0.95))
        plt.colorbar(label="Déplacement [nm]")

        ### Affichage des tags
        for (date, hour, text) in tags:
            # plt.scatter(np.datetime64(date), hour, zorder=50, s=40, color="red", edgecolors="white")
            plt.axvline(np.datetime64(date), zorder=50, color="white", ls="-")
            plt.text(np.datetime64(date)-8, 0.75, text, color="white", fontsize="medium", rotation=90)   # Confinement 1

        ### Configuration de l'axe vertical
        plt.ylabel("Heure locale")
        y_ticks = ["%ih" % j for j in range(1, 24)]
        y_loc = [j for j in range(1, 24)]
        plt.yticks(y_loc, y_ticks)
        plt.title("{} - [{}] Hz".format(st_str, freq))
        folder_path = FIG_FOLDER + "/fr_hourly_variations/" + st_str

        if not os.path.exists(folder_path): os.makedirs(folder_path)
        plt.savefig("{}/hourly_temporal_{}.png".format(folder_path, freq), dpi=150)
        plt.close()
        plt.clf()
        
    k += 1

1/1    G.CCD.00.BHZ


<Figure size 432x288 with 0 Axes>

## 2.2 Calcul de variations horaires sur plusieurs sous-périodes

In [7]:
subperiods = [
    ["2020-10-01", "2020-10-20", "No lockdown", "#8F8F8F"],
    ["2020-03-18", "2020-05-10", "Lockdown 1", "#3D80B0"],
    ["2020-11-01", "2020-12-10", "Lockdown 2", "#C53F2C"],
    ["2020-12-10", "2020-12-31", "Christmas + Curfew 20h-6h", "#299739"],
    ["2021-01-16", "2021-02-16", "Curfew 18h-6h", "#902F8C"],
    ["2021-04-03", "2021-05-03", "Lockdown 3", "#E59B1A"],
    ["2021-05-03", "2021-06-09", "Crufew 19h", "green"],
    ["2021-06-09", "2021-06-30", "Crufew 23h", "blue"],
    ["2021-06-30", "2021-07-31", "No curfew", "red"]
]

In [21]:
k = 1

for st_str in st_name:
    print("{}/{}    {}".format(k, len(st_name), st_str))
    for freq in dRMS[st_str].columns:

        try:
            noise = dRMS[st_str][freq]  # Série temporelle du bruit
        except:
            continue

        t, h, noise_matrix = hourly_var(noise, periods)
        grid = noise_matrix * 1e9  # Données en [nm]
        
        # h = np.append(h[8:], h[:8])

        ### Calcul du bruit pour que sous-période
        plt.figure(figsize=(12, 6))
        for i, value in enumerate(subperiods):
            time1 = np.datetime64(subperiods[i][0])
            time2 = np.datetime64(subperiods[i][1])
            color = subperiods[i][3]
            label = "{} to {} : {}".format(subperiods[i][0], subperiods[i][1], subperiods[i][2])
            k1 = np.where(t == time1)[0][0]
            k2 = np.where(t == time2)[0][0]

            median = np.nanmedian(grid[:,k1:k2], axis=1)
            median = np.append(median[8:], median[:8])
            
            if subperiods[i][2]=="No lockdown": plt.plot(h, median, label=label, color=color, lw=2, ls="--")
            else: plt.plot(h, median, label=label, color=color, lw=2)

        ### Mise en graphique
        plt.tick_params(direction="in", which="both", top=True, right=True)
        plt.title("{} - [{}] Hz".format(st_str,freq))
        plt.grid(color="#D5D5D5", linestyle='--', linewidth=1)
        # plt.ylabel("Median displacement [nm]")
        plt.xlim([0, 23.5])
        h_tick = np.append(np.arange(4, 23, 2), np.arange(0, 4, 2))
        h_label = ["4h","6h","8h","10h","12h","14h","16h","18h","20h","22h", "0h", "2h"]
        plt.xticks(np.arange(0, 23, 2), h_label)
        # plt.xticks(np.arange(0, 23, 2), ["0h","2h","4h","6h","8h","10h","12h","14h","16h","18h","20h","22h"])

        plt.legend(loc="upper left",   # Position of legend
                   borderaxespad=0.2,    # Small spacing around legend box
                   bbox_to_anchor=(-0.1, -0.1),
                   ncol=3)

        plt.tight_layout()

        folder_path = FIG_FOLDER + "/fr_hourly_variations/" + st_str

        if not os.path.exists(folder_path): os.makedirs(folder_path)

        plt.savefig("{}/hourly_periods_{}.png".format(folder_path, freq), dpi=150)
        plt.close()
        plt.clf()

    k += 1

1/130    FR.ABJF.00.HHZ
2/130    FR.AGO.00.HHZ
3/130    FR.AJAC.00.HHZ
4/130    FR.ARBF.00.HHZ
5/130    FR.ARTF.00.HHZ
6/130    FR.ATE.00.HHZ
7/130    FR.BALS.00.HHZ
8/130    FR.BANN.00.HHZ
9/130    FR.BARI.00.HHZ
10/130    FR.BEGF.00.HHZ
11/130    FR.BLAF.00.HHZ
12/130    FR.BOUC.00.HHZ
13/130    FR.BSTF.00.HHZ
14/130    FR.CALF.00.HHZ
15/130    FR.CAMF.00.HHZ
16/130    FR.CARF.00.HHZ
17/130    FR.CFF.00.HHZ
18/130    FR.CHIF.00.HHZ
19/130    FR.CHMF.00.HHZ
20/130    FR.CIEL.00.HHZ
21/130    FR.CLAF.00.HHZ
22/130    FR.CLF.00.HHZ
23/130    FR.CLMF.00.HHZ
24/130    FR.COLF.00.HHZ
25/130    FR.CRAS.00.HHZ
26/130    FR.CRNF.00.HHZ
27/130    FR.CURIE.00.HHZ
28/130    FR.DOU.00.HHZ
29/130    FR.DUNF.00.HHZ
30/130    FR.EILF.00.HHZ
31/130    FR.ENAUX.00.HHZ
32/130    FR.ESCA.01.HHZ
33/130    FR.FAHY.00.HHZ
34/130    FR.FILF.00.HHZ
35/130    FR.FNEB.00.HHZ
36/130    FR.FOURG.00.HHZ
37/130    FR.FRNF.00.HHZ
38/130    FR.GENF.00.HHZ
39/130    FR.GIZF.00.HHZ
40/130    FR.GRN.00.HHZ
41/130    FR

<Figure size 432x288 with 0 Axes>